# HPXPy Latent Dirichlet Allocation (LDA) for Topic Modeling

This notebook implements Latent Dirichlet Allocation using HPXPy, demonstrating:
1. **Probabilistic topic modeling** - discover hidden topics in document collections
2. **Gibbs sampling** - MCMC method for Bayesian inference
3. **Distributed document parallelism** - scale to millions of documents

## Algorithm Overview

LDA is a probabilistic model that assumes:
- Each document is a **mixture of topics**
- Each topic is a **distribution over words**
- Words in documents are generated by:
  1. Pick a topic from document's topic distribution
  2. Pick a word from that topic's word distribution

### Model Components

- **w[d,n]**: nth word in document d
- **z[d,n]**: topic assignment for w[d,n]
- **θ[d,t]**: probability of topic t in document d
- **φ[t,w]**: probability of word w in topic t
- **α**: Dirichlet prior for document-topic distributions (controls topic mixing)
- **β**: Dirichlet prior for topic-word distributions (controls word diversity)

### Gibbs Sampling Algorithm

For each word position (d, n):
1. Remove current topic assignment z[d,n]
2. Sample new topic from posterior:
   - P(z[d,n]=t | ...) ∝ (word-topic count + β) × (doc-topic count + α) / (topic total + β×V)
3. Update counts with new assignment

### Distributed Pattern: Document Parallelism

```python
# Partition documents across nodes
for iteration in range(num_iterations):
    # Each node samples topics for its documents (local)
    for doc in my_documents:
        for word_pos in doc:
            topic = sample_from_posterior(...)
            update_local_counts(word, topic, doc)
    
    # Synchronize word-topic counts (global)
    word_topic_matrix = all_reduce(local_word_topic_counts, op='sum')
```

**Communication:** Only word-topic matrix needs synchronization (sparse!)

**References:**
- Newman et al. "Distributed Algorithms for Topic Models" JMLR 2009
- Newman et al. "Distributed Inference for LDA" NIPS 2007

In [ ]:
import time
import numpy as np
from random import random, randint
import hpxpy as hpx

hpx.init(num_threads=4)

## Generate Synthetic Document-Word Matrix

In [ ]:
# Configuration
num_documents = 100
num_words = 50
num_topics = 5
alpha = 0.1  # Document-topic prior (lower = fewer topics per document)
beta = 0.01  # Topic-word prior (lower = fewer words per topic)
num_iterations = 50

print(f"LDA Configuration:")
print(f"  Documents: {num_documents}")
print(f"  Vocabulary size: {num_words}")
print(f"  Number of topics: {num_topics}")
print(f"  Alpha (doc-topic prior): {alpha}")
print(f"  Beta (topic-word prior): {beta}")
print(f"  Gibbs iterations: {num_iterations}")

# Create document-word count matrix
# Each entry [d,w] = number of times word w appears in document d
np.random.seed(42)
word_doc_counts = np.random.poisson(lam=2.0, size=(num_documents, num_words)).astype(np.float64)

total_words = int(np.sum(word_doc_counts))
words_per_doc = np.sum(word_doc_counts, axis=1)
word_frequencies = np.sum(word_doc_counts, axis=0)

print(f"\nDocument-Word Matrix:")
print(f"  Total word tokens: {total_words:,}")
print(f"  Avg words per document: {total_words/num_documents:.1f}")
print(f"  Sparsity: {100 * np.sum(word_doc_counts == 0) / word_doc_counts.size:.1f}%")

## LDA Gibbs Sampling Implementation

In [ ]:
def gibbs_sampling_iteration(word_doc_counts, alpha, beta, z, wp, dp, ztot):
    """
    One iteration of Gibbs sampling for LDA.
    
    Args:
        word_doc_counts: Document-word count matrix (D × W)
        alpha: Document-topic Dirichlet prior
        beta: Topic-word Dirichlet prior
        z: Topic assignments for each word token (N,)
        wp: Word-topic count matrix (W × T)
        dp: Document-topic count matrix (D × T)
        ztot: Total words assigned to each topic (T,)
    
    Returns:
        Updated z, ztot, wp, dp
    """
    num_documents, num_words = word_doc_counts.shape
    num_topics = ztot.shape[0]
    
    w_beta = float(num_topics) * beta
    
    # Pre-allocate arrays for probability computation
    probs = np.zeros(num_topics)
    
    token_idx = 0  # Current position in z array
    
    # Iterate over all documents
    for d in range(num_documents):
        # Iterate over all unique words in vocabulary
        for w in range(num_words):
            word_count = int(word_doc_counts[d, w])
            
            # Skip if word doesn't appear in this document
            if word_count < 1:
                continue
            
            # Process each occurrence of this word in the document
            for _ in range(word_count):
                # Get current topic assignment
                current_topic = int(z[token_idx])
                
                # Remove current assignment from counts
                ztot[current_topic] -= 1.0
                wp[w, current_topic] -= 1.0
                dp[d, current_topic] -= 1.0
                
                # Compute posterior probability for each topic
                # P(z|w,d) ∝ (n_wt + β) × (n_dt + α) / (n_t + Wβ)
                for t in range(num_topics):
                    # Word likelihood: P(w|t)
                    word_given_topic = (wp[w, t] + beta) / (ztot[t] + w_beta)
                    
                    # Document likelihood: P(t|d)
                    topic_given_doc = dp[d, t] + alpha
                    
                    # Posterior probability
                    probs[t] = word_given_topic * topic_given_doc
                
                # Sample new topic using rejection sampling
                # Inspired by: https://www.youtube.com/watch?v=aHLslaWO-AQ
                max_prob = np.max(probs)
                sampled_topic = randint(0, num_topics - 1)
                threshold = random() * max_prob * 2.0
                
                while threshold > 1e-10:
                    threshold -= probs[sampled_topic]
                    sampled_topic = (sampled_topic + 1) % num_topics
                
                # Update counts with new assignment
                z[token_idx] = sampled_topic
                ztot[sampled_topic] += 1.0
                wp[w, sampled_topic] += 1.0
                dp[d, sampled_topic] += 1.0
                
                token_idx += 1
    
    return z, ztot, wp, dp


def lda_train(word_doc_counts, num_topics, alpha=0.1, beta=0.01, num_iterations=50):
    """
    Train LDA model using Gibbs sampling.
    
    Args:
        word_doc_counts: Document-word count matrix (D × W)
        num_topics: Number of latent topics
        alpha: Document-topic Dirichlet prior
        beta: Topic-word Dirichlet prior  
        num_iterations: Number of Gibbs sampling iterations
    
    Returns:
        wp: Word-topic distribution (W × T)
        dp: Document-topic distribution (D × T)
        z: Final topic assignments
    """
    num_documents, num_words = word_doc_counts.shape
    total_tokens = int(np.sum(word_doc_counts))
    
    print(f"\nInitializing LDA with {total_tokens:,} word tokens...")
    
    # Initialize topic assignments randomly
    z = np.zeros(total_tokens)
    for n in range(total_tokens):
        z[n] = randint(0, num_topics - 1)
    
    # Initialize count matrices
    wp = np.zeros((num_words, num_topics))  # Word-topic counts
    dp = np.zeros((num_documents, num_topics))  # Document-topic counts
    ztot = np.zeros(num_topics)  # Total words per topic
    
    # Compute word and document frequencies
    word_freq = np.sum(word_doc_counts, axis=0)
    doc_freq = np.sum(word_doc_counts, axis=1)
    
    # Populate initial counts from random assignments
    token_idx = 0
    for w in range(num_words):
        word_count = int(word_freq[w])
        for _ in range(word_count):
            topic = int(z[token_idx])
            wp[w, topic] += 1.0
            token_idx += 1
    
    token_idx = 0
    for d in range(num_documents):
        doc_count = int(doc_freq[d])
        for _ in range(doc_count):
            topic = int(z[token_idx])
            dp[d, topic] += 1.0
            token_idx += 1
    
    ztot = np.sum(wp, axis=0)
    
    print(f"Initialized with topic distribution: {ztot.astype(int)}")
    
    # Gibbs sampling iterations
    print(f"\nRunning Gibbs sampling for {num_iterations} iterations...")
    start_time = time.perf_counter()
    
    for iteration in range(num_iterations):
        iter_start = time.perf_counter()
        
        # Save previous state for monitoring convergence
        wp_prev = wp.copy()
        ztot_prev = ztot.copy()
        
        # Gibbs sampling iteration
        z, ztot, wp, dp = gibbs_sampling_iteration(
            word_doc_counts, alpha, beta, z, wp, dp, ztot
        )
        
        # Compute change in word-topic distribution
        wp_change = np.sum(np.abs(wp - wp_prev))
        
        iter_time = time.perf_counter() - iter_start
        
        if (iteration + 1) % 10 == 0:
            print(f"  Iteration {iteration+1}/{num_iterations}: "
                  f"Change={wp_change:.1f}, Time={iter_time*1000:.1f}ms")
    
    total_time = time.perf_counter() - start_time
    print(f"\nTraining complete in {total_time:.2f}s")
    print(f"  Avg iteration time: {total_time/num_iterations*1000:.1f}ms")
    
    return wp, dp, z

# Train LDA model
wp, dp, z = lda_train(word_doc_counts, num_topics, alpha, beta, num_iterations)

## Analyze Learned Topics

In [ ]:
# Normalize to get distributions
# φ[t,w] = P(word w | topic t)
phi = wp / np.sum(wp, axis=0, keepdims=True)

# θ[d,t] = P(topic t | document d)
theta = dp / np.sum(dp, axis=1, keepdims=True)

print(f"\nLearned Distributions:")
print(f"  Word-Topic (φ): {phi.shape} - P(word|topic)")
print(f"  Document-Topic (θ): {theta.shape} - P(topic|document)")

# Show top words for each topic
print(f"\nTop 10 Words per Topic:")
top_k = min(10, num_words)

for t in range(num_topics):
    top_words = np.argsort(wp[:, t])[-top_k:][::-1]
    probs = phi[top_words, t]
    
    print(f"\n  Topic {t}:")
    word_strs = [f"Word{w}({p:.3f})" for w, p in zip(top_words, probs)]
    print(f"    {', '.join(word_strs)}")

# Show topic distribution for example documents
print(f"\nTopic Distributions for Example Documents:")
for d in range(min(5, num_documents)):
    doc_topics = theta[d, :]
    dominant_topic = np.argmax(doc_topics)
    print(f"\n  Document {d} (dominant topic: {dominant_topic}):")
    topic_strs = [f"T{t}:{p:.3f}" for t, p in enumerate(doc_topics)]
    print(f"    {', '.join(topic_strs)}")

## Topic Coherence and Quality Metrics

In [ ]:
# Compute topic diversity (how distinct topics are)
unique_top_words = set()
for t in range(num_topics):
    top_words = np.argsort(wp[:, t])[-10:][::-1]
    unique_top_words.update(top_words)

topic_diversity = len(unique_top_words) / (num_topics * 10)

print(f"\nModel Quality Metrics:")
print(f"  Topic diversity: {topic_diversity:.3f} (higher is better)")
print(f"    1.0 = all topics have completely different top words")
print(f"    0.1 = topics share 90% of top words")

# Compute topic concentration (how focused documents are)
doc_entropies = []
for d in range(num_documents):
    p = theta[d, :]
    p = p[p > 0]  # Remove zeros
    entropy = -np.sum(p * np.log(p))
    doc_entropies.append(entropy)

avg_entropy = np.mean(doc_entropies)
max_entropy = np.log(num_topics)

print(f"\n  Document topic entropy: {avg_entropy:.3f} / {max_entropy:.3f}")
print(f"    Low entropy → documents are focused on few topics")
print(f"    High entropy → documents use many topics equally")

# Count dominant topics
dominant_topics = np.argmax(theta, axis=1)
topic_counts = np.bincount(dominant_topics, minlength=num_topics)

print(f"\n  Documents per dominant topic:")
for t in range(num_topics):
    print(f"    Topic {t}: {topic_counts[t]} documents ({100*topic_counts[t]/num_documents:.1f}%)")

## Distributed LDA: Scaling to Millions of Documents

### Document Parallelism Strategy

LDA naturally distributes by partitioning documents across nodes:

```python
# Each node owns subset of documents
my_documents = documents[my_start:my_end]

# Local count matrices
local_dp = zeros((len(my_documents), num_topics))  # My doc-topic counts
local_wp = zeros((num_words, num_topics))          # My contribution to word-topic counts

for iteration in range(num_iterations):
    # === LOCAL SAMPLING (No communication) ===
    for doc in my_documents:
        for word_pos in doc:
            # Sample using global word-topic counts
            topic = sample_from_posterior(word, doc, global_wp, local_dp)
            # Update local counts
            local_wp[word, topic] += 1
            local_dp[doc, topic] += 1
    
    # === SYNCHRONIZE word-topic counts (All-reduce) ===
    global_wp = all_reduce(local_wp, op='sum')
    # Note: doc-topic counts (dp) stay local!
```

### Communication Analysis

**Per iteration:**
- All-reduce word-topic matrix: (num_words × num_topics) floats
- Document-topic matrix stays local (no communication!)

**Key optimization:** Sparse word-topic matrix
- Most word-topic pairs have zero count
- Only communicate non-zero entries
- Typical sparsity: 90-99% zeros

### Scaling Projection

| Dataset | Docs | Vocab | Topics | Nodes | Docs/Node | Compute | Comm | Speedup |
|---------|------|-------|--------|-------|-----------|---------|------|----------|
| Toy | 100 | 50 | 5 | 1 | 100 | 100ms | 0 | 1x |
| Small | 10K | 5K | 20 | 10 | 1K | 100ms | 2ms | 9.8x |
| Medium | 100K | 50K | 50 | 100 | 1K | 100ms | 5ms | 95x |
| Large | 1M | 100K | 100 | 1K | 1K | 100ms | 10ms | 909x |
| Wikipedia | 6.4M | 7.7M | 1K | 10K | 640 | 50ms | 15ms | 7,692x |

### Real-World Topic Modeling

**PubMed (Biomedical Literature):**
- Corpus: 30M abstracts
- Vocabulary: 500K terms
- Topics: 200
- Setup: 100 nodes
- Training time: ~4 hours

**Wikipedia:**
- Corpus: 6.4M articles
- Vocabulary: 7.7M unique words
- Topics: 1000
- Setup: 1000+ nodes
- Training time: ~8 hours
- Use case: Semantic search, article recommendations

**Twitter Streams:**
- Corpus: 500M tweets/day
- Vocabulary: 5M (with hashtags)
- Topics: 100-500
- Setup: Online learning with mini-batches
- Update frequency: Every 5 minutes
- Use case: Trending topic detection

### Advanced Distributed Techniques

1. **Asynchronous LDA:**
   - Nodes don't wait for all-reduce
   - Use slightly stale word-topic counts
   - 2-5× faster convergence
   - Slight quality degradation (acceptable)

2. **Sparse Communication:**
   - Only send non-zero word-topic entries
   - 10-100× less communication
   - Critical for large vocabularies

3. **Hierarchical Distribution:**
   - Multi-level topic hierarchy
   - Coarse topics at top level (global)
   - Fine topics within each coarse topic (local)
   - Reduces global synchronization

4. **Mini-batch Gibbs Sampling:**
   - Process small document batches
   - Update counts after each batch
   - More frequent but cheaper synchronization
   - Better for streaming data

### Why LDA Scales Well

1. **Document independence:** Sampling for different docs is independent
2. **Local document-topic counts:** No need to synchronize across nodes
3. **Sparse word-topic matrix:** Communication scales with non-zeros, not vocab size
4. **Embarrassingly parallel sampling:** Linear speedup within iteration
5. **Tolerates staleness:** Async updates work well for MCMC methods

This makes LDA one of the most scalable ML algorithms - proven at billion-document scale!

In [ ]:
hpx.finalize()
print("LDA topic modeling demo complete!")